In [ ]:

import pandas as pd

In [ ]:
df = pd.read_csv('./financialNews.csv', encoding='latin-1')

In [ ]:
df

In [ ]:
df.columns = ['Sentiment', 'SentimentText']

In [ ]:
df

In [ ]:
mapper = {'negative': 0,
         'neutral': 1,
         'positive': 2,}
df.Sentiment = df.Sentiment.map(mapper)

In [ ]:
df

In [ ]:
df.Sentiment.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import keras

train, valid = train_test_split(df, test_size=0.2)

In [ ]:
train_text = np.array(train['SentimentText'].tolist().copy())
labels = keras.utils.to_categorical(train['Sentiment'].astype('int64'))

In [ ]:
valid_text = np.array(valid['SentimentText'].tolist().copy())
labels_valid = keras.utils.to_categorical(valid['Sentiment'].astype('int64'))

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#Set up variables for preprocessing and learning
vocab_size = 1000
embedding_dim = 16
max_length = 142
trunc_type='post'
padding_type='post'
oov_token = '<OOV>'

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_text)

# tokenizer.word_index

sequences = tokenizer.texts_to_sequences(train_text)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(valid_text)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


In [ ]:
import pickle

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
num_epochs = 30
history = model.fit(padded, labels, epochs=num_epochs, validation_data=(testing_padded, labels_valid))

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

#Set up variables for preprocessing and learning
vocab_size = 1000
embedding_dim = 16
max_length = 142
trunc_type='post'
padding_type='post'
oov_token = '<OOV>'

phrase = ['TSLA will compound by 10% over the next 10 years']

testing_sequences = tokenizer.texts_to_sequences(phrase)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length,
                              padding=padding_type,truncating=trunc_type)

pred = model.predict(testing_padded)
classes = np.argmax(pred, axis=-1)
dict_sentiment = {0:'Negative', 1:'Neutral', 2: 'Positive'}
print(f'{phrase} : {dict_sentiment[int(classes)]}')

['TSLA will compound by 10% over the next 10 years'] : Positive


In [ ]:
#Save Model

model.save('model1.h5')

In [6]:
import tensorflow as tf
import pickle
model = tf.keras.models.load_model('model1.h5')

#Load tokenizer
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)